In [1]:
from langchain_community.document_loaders import PyPDFLoader
from plum_chatbot.configs.folders import PDF_DIR


from pypdf import PdfReader


In [2]:
pages = []
files = list(filter(lambda x: "merged" not in str(x), PDF_DIR.glob("*.pdf")))

for document in files:
    loader = PyPDFLoader(PDF_DIR / document)
    content = ""
    async for page in loader.alazy_load():
        # pages.append(page)
        content += page.page_content + "\n"
    pages.append(content)
pages.sort(key = lambda x: int(x.split(".")[0]))

In [3]:
pages

['1. COME CONFIGUARE LE EMAIL E LA PEC\nAll’interno del portale è possibile configurare tutti gli indirizzi email e Pec dello\nStudio  e  decidere,  a  seconda  delle  esigenze,  da  quale  indirizzo  inviare  la\ncorrispondenza.  \nDal menu principale cliccare su Configurazioni \uf0e0 Parametri Email \nPremere ora il simbolo \n  per inserire i parametri dell’email \nCompilare i campi della finestra :\n- Descrizione  (inserire  il  riferimento  dell’email,  es:  Mario  oppure  Solleciti\noppure amministrazione, ecc.)\n- Tipo  (inserire per le email ‘ORDINARIA’ e per la pec ‘PEC’)\n- Indirizzo (riportare  l’indirizzo  dell’email  o  della  pec,  es:\nammistrazionerossi@gmail.com oppure info@studioverdi.it )\n- Host (selezionare dall’elenco il provider di posta elettronica. Se non presente\nnella lista contattare l’assistenza PLUM).\n- Username (generalmente l’indirizzo email già inserito nel campo ‘Indirizzo’).\n- Password (inserire la password delle propria email o pec).\n- Mail inoltr

In [17]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    # separators=["\n\n", "\n", ".", " "],
    separators=["\n", "."],
)

In [20]:
from uuid import uuid4

all_chunks = []
for document in files:
    loader = PyPDFLoader(document)
    docs = loader.load()
    splits = text_splitter.split_documents(docs)  # Proper usage
    for i, split in enumerate(splits):
        all_chunks.append({
            "id": str(uuid4()),
            "text": split.page_content,
            "metadata": {
                "source": document.name,
                "chunk_index": i,
                "title": document.name.replace(".pdf", "")
            }
        })

In [22]:
all_chunks

[{'id': '150797e1-9dcd-4986-af3c-366d9139c1a7',
  'text': '4. COME IMPOSTARE LE COORDINATE DEI BOLLETTINI 123\nAll’interno  del  portale  è  possibile  configurare  impostare  le  coordinate  dei\nbollettini 123.  \nDal menu principale cliccare su Configurazioni \uf0e0 Parametri Bollettino 123 \nLa procedura visualizza la finestra per inserire le coordinate dei bollettini 123 \nPremere il pulsante SALVA per memorizzare i dati.',
  'metadata': {'source': '4. COME INSERIRE I PARAMETRI DEI BOLLETTINI 123.pdf',
   'chunk_index': 0,
   'title': '4. COME INSERIRE I PARAMETRI DEI BOLLETTINI 123'}},
 {'id': '8b9d23fe-89d3-48d5-9894-516186e16325',
  'text': 'Compilare i campi della finestra :\n- Descrizione  (inserire  il  riferimento  dell’email,  es:  Mario  oppure  Solleciti\noppure amministrazione, ecc.)\n- Tipo  (inserire per le email ‘ORDINARIA’ e per la pec ‘PEC’)\n- Indirizzo (riportare  l’indirizzo  dell’email  o  della  pec,  es:\nammistrazionerossi@gmail.com oppure info@studioverdi.i

In [23]:
from qdrant_client import QdrantClient, models
import ollama

from plum_chatbot.configs.settings import Settings

settings = Settings()

COLLECTION_NAME = "FAQ2"

# Initialize Ollama client
oclient = ollama.Client(host="host.docker.internal")

# Initialize Qdrant client
qclient = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)


/tmp/ipykernel_41724/247937773.py:14: UserWarning: Api key is used with an insecure connection.
  qclient = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)


In [29]:
# Text to embed

# Generate embeddings
# page_contents = [document.page_content for document in pages]
# responses = [oclient.embeddings(model="llama3.2", prompt=content) for content in pages]
responses = [oclient.embeddings(model="llama3.2", prompt=content['text']) for content in all_chunks]
embeddings = [response["embedding"] for response in responses]

# Create a collection if it doesn't already exist
if not qclient.collection_exists(COLLECTION_NAME):
    qclient.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=len(embeddings[0]), distance=models.Distance.COSINE
        ),
    )


In [30]:
# Upload the vectors to the collection along with the original text as payload

qclient.upsert(
    collection_name=COLLECTION_NAME,
    points=[
        models.PointStruct(id=i, vector=embedding, payload=chunk)
        for i, (chunk, embedding) in enumerate(zip(all_chunks, embeddings))
        if embedding
    ],
)


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)